# 分析
切り出したショットデータを取得し、分析します。  
以下は特定ショットのデータを取得/確認し、特徴点（荷重開始点/最大荷重点/破断点）を抽出、その後全ショットから特徴点を抽出するサンプルコードです。抽出した特徴点はElasticsearchに格納されます。

In [1]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from data_reader.data_reader import DataReader
from elastic_manager.elastic_manager import ElasticManager
sys.path.append(os.path.join(os.getcwd(), '../src/analyze/'))
from analyze.h_one_extract_features import *
from analyze.analyze import *

Loading BokehJS ...

In [2]:
# ログ設定
LOG_FILE = os.path.join(
    common.get_config_value(common.APP_CONFIG_PATH, "log_dir"), "analyze/analyze.log"
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.handlers.RotatingFileHandler(
            LOG_FILE, maxBytes=common.MAX_LOG_SIZE, backupCount=common.BACKUP_COUNT
        ),
        logging.StreamHandler(),
    ],
)

In [3]:
# ショットデータインデックス一覧
ElasticManager.show_indices(index="shots-*-data")

,index,docs.count,store.size
0,shots-20201201010000-data,9356061,1178983
1,shots-20210101000000-data,4040,751
2,shots-20210327141514-data,13586763,1888984


In [3]:
# 対象データ設定
#target = "20201201010000"
target = "20210327141514"
shots_data_index = "shots-" + target + "-data"
shots_meta_index = "shots-" + target + "-meta"

# 1ショットで特徴点抽出

## ショット1番を取得して確認

In [5]:
dr = DataReader()

shot_number = 10
shot_df = dr.read_shot(shots_data_index, shot_number=shot_number)
shot_df.head()

,timestamp,sequential_number,sequential_number_by_shot,rawdata_sequential_number,displacement,load01,load02,load03,load04,shot_number,tags
0,1.616790e+09,179065,0,11201752,50.61000,-0.044,-0.014,0.034,0.025,10,[]
1,1.616790e+09,179066,1,11201753,50.61000,-0.041,-0.008,0.029,0.026,10,[]
2,1.616790e+09,179067,2,11201754,50.61000,-0.047,-0.013,0.021,0.022,10,[]
3,1.616790e+09,179068,3,11201755,50.60125,-0.045,-0.013,0.020,0.022,10,[]
4,1.616790e+09,179069,4,11201756,50.59250,-0.048,-0.020,0.013,0.013,10,[]


In [6]:
# ショットメタデータを確認
shots_meta_df = dr.read_shots_meta(shots_meta_index)
shots_meta_df.head()

,shot_number,spm,num_of_samples_in_cut_out
0,1,50.847458,109404.0
1,2,NaN,16564.0
2,3,28.571429,9877.0
3,4,32.608696,8640.0
4,5,35.928144,7798.0


In [7]:
# ショットの数を確認
print(f"ショット数：{len(shots_meta_df)}")

ショット数：3260


In [ ]:
pd.set_option('display.max_rows', 3300)
shots_meta_df[shots_meta_df.spm > 80]

In [ ]:
#import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

step = 300
for i in range(0, 3300, step):
    x = shots_meta_df.shot_number[i:i+step]
    y = shots_meta_df.spm[i:i+step]
    plt.xlabel("shot_number")
    plt.ylabel("spm")
#    fig = plt.figure()
    plt.plot(x, y)
    plt.show()
#    plt.savefig(f"shots-{i}-{i+step}")

In [ ]:
# spm
spm = float(shots_meta_df[shots_meta_df.shot_number == shot_number].spm)
print(f"spm：{spm}")

## 特徴点抽出
1ショットから最大荷重点、荷重開始点、破断点を取得します。  

***shot_data***: 対象ショットデータ  
***spm***: spm  
***func***：適用するロジック  
***sub_func***：サブロジック  
***disp_chart***: グラフ表示

In [ ]:
# 最初のショットの最大荷重点、荷重開始点、破断点を確認
max_load_indicies, max_load_values, max_debug_values = extract_features(shot_data=shot_df, spm=spm, func=max_load, disp_chart=True)
load_start_indicies, load_start_values, start_debug_values = extract_features(shot_data=shot_df, spm=spm, func=load_start3, disp_chart=True)
break_load_indicies, break_load_values, break_debug_values = extract_features(shot_data=shot_df, spm=spm, func=breaking_var_vrms, disp_chart=True)

In [ ]:
# 各特徴点のインデックス番号と値を確認
print(f"【最大荷重点】\n indicies：{max_load_indicies}, values：{max_load_values}")
print(f"【荷重開始点】\n indicies：{load_start_indicies}, values：{load_start_values}")
print(f"【破断点】　　\n indicies：{break_load_indicies}, values：{break_load_values}")

# 全ショットから特徴点抽出
全ショットから最大荷重点、荷重開始点、破断点を抽出し、Elasticsearchに保存します。  

***target***: 対象ショットデータ（yyyyMMddHHMMSS）   
***feature***: 抽出する特徴点名（max/start/break）  
***func***：適用するロジック  
***sub_func***：サブロジック  
***exclude_shots***: 除外ショット(ex: exclude_shots=(1227, 1228, 1229))  

保存先インデックスは以下になります。  
* 最大荷重点：shots-yyyyMMddHHMMSS-max-point
* 荷重開始点：shots-yyyyMMddHHMMSS-start-point
* 破断点：shots-yyyyMMddHHMMSS-break-point

In [4]:
# 最大荷重点
apply(
    target=target,
    feature="max",
    func=max_load,
    sub_func=None,
)

2021-04-16 08:36:59,474 [INFO] apply start.
2021-04-16 08:37:47,937 [INFO] shot_number: 2894 not found.
2021-04-16 08:38:03,077 [INFO] shot_number: 983 not found.
2021-04-16 08:38:10,709 [INFO] shot_number: 1369 not found.
2021-04-16 08:38:10,938 [INFO] shot_number: 1381 not found.
2021-04-16 08:38:20,240 [INFO] apply finished.


In [5]:
dr = DataReader()
max_index = "shots-" + target + "-max-point"
max_df = dr.read_all(max_index)
max_df

2021-04-16 08:39:25,356 [INFO] データを全件取得します。データ件数が多い場合、長時間かかる場合があります。
2021-04-16 08:39:25,360 [INFO] データ件数: 13040
2021-04-16 08:39:25,456 [INFO] Data reading has finished.


,shot_number,load,sequential_number,sequential_number_by_shot,value
0,1,load04,4943,4943,0.155258
1,1,load01,5712,5712,0.283157
2,1,load02,5950,5950,0.127714
3,1,load03,6271,6271,0.339230
4,2,load01,117929,8525,1.159921
...,...,...,...,...,...
13035,3263,load01,13574938,5260,1.905189
13036,3264,load03,13585120,4964,1.184347
13037,3264,load04,13585179,5023,1.224862
13038,3264,load01,13585724,5568,1.714805


In [6]:
# 荷重開始点
apply(
    target=target,
    feature="start",
    func=load_start3,
    sub_func=None,
    exclude_shots=(1227,1228,1229)
)

2021-04-16 08:39:29,500 [INFO] apply start.
2021-04-16 08:39:29,601 [INFO] delete index 'shots-20210327141514-start-point' finished. result: {'acknowledged': True}
2021-04-16 08:40:14,591 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40:18,014 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40:19,768 [INFO] shot_number: 2894 not found.
2021-04-16 08:40:28,803 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40:30,414 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40:32,532 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40:33,503 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40:38,439 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40:38,576 [INFO] shot_number: 1227 was excluded.
2021-04-16 08:40:38,578 [INFO] shot_number: 1228 was excluded.
2021-04-16 08:40:38,580 [INFO] shot_number: 1229 was excluded.
2021-04-16 08:40:38,647 [INFO] shot_number: 983 not found.
2021-04-16 08:40:43,033 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:40

In [7]:
dr = DataReader()
start_index = "shots-" + target + "-start-point"
start_df = dr.read_all(start_index)
start_df

2021-04-16 08:41:15,105 [INFO] データを全件取得します。データ件数が多い場合、長時間かかる場合があります。
2021-04-16 08:41:15,108 [INFO] データ件数: 13028
2021-04-16 08:41:15,259 [INFO] Data reading has finished.


,shot_number,load,sequential_number,sequential_number_by_shot,value
0,1,load03,2455,2455,-0.011168
1,1,load01,2494,2494,0.001193
2,1,load02,2631,2631,-0.001962
3,1,load04,2644,2644,-0.004366
4,2,load04,113772,4368,0.004363
...,...,...,...,...,...
13023,3263,load02,13571135,1457,-0.009293
13024,3264,load01,13581565,1409,-0.018143
13025,3264,load03,13581596,1440,-0.044704
13026,3264,load04,13581641,1485,-0.027687


In [8]:
# 破断点
apply(
    target=target,
    feature="break",
    func=breaking_vmin_amin,
    sub_func=narrowing_v4min_mab,
    exclude_shots=(1227,1228,1229)    
)

2021-04-16 08:41:21,117 [INFO] apply start.
2021-04-16 08:41:21,184 [INFO] delete index 'shots-20210327141514-break-point' finished. result: {'acknowledged': True}
2021-04-16 08:42:06,647 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:42:10,101 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:42:12,080 [INFO] shot_number: 2894 not found.
2021-04-16 08:42:21,960 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:42:22,962 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:42:23,758 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:42:25,424 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:42:27,315 [INFO] shot_number: 983 not found.
2021-04-16 08:42:30,025 [INFO] NumExpr defaulting to 8 threads.
2021-04-16 08:42:30,099 [INFO] shot_number: 1227 was excluded.
2021-04-16 08:42:30,101 [INFO] shot_number: 1228 was excluded.
2021-04-16 08:42:30,104 [INFO] shot_number: 1229 was excluded.
2021-04-16 08:42:34,361 [INFO] shot_number: 1369 not found.
2021-04-16 08:42:34,

In [9]:
dr = DataReader()
break_index = "shots-" + target + "-break-point"
break_df = dr.read_all(break_index)
break_df

2021-04-16 08:43:02,694 [INFO] データを全件取得します。データ件数が多い場合、長時間かかる場合があります。
2021-04-16 08:43:02,697 [INFO] データ件数: 13028
2021-04-16 08:43:02,812 [INFO] Data reading has finished.


,shot_number,load,sequential_number,sequential_number_by_shot,value,break_channels
0,1,load01,6266,6266,0.276385,"[load01, load02]"
1,1,load02,6266,6266,0.128082,"[load01, load02]"
2,1,load03,6266,6266,0.348574,"[load01, load02]"
3,1,load04,6266,6266,0.162974,"[load01, load02]"
4,2,load04,119010,9606,1.230244,"[load01, load02]"
...,...,...,...,...,...,...
13023,3263,load04,13575032,5354,-0.028044,"[load03, load04]"
13024,3264,load02,13585772,5616,1.317613,"[load03, load04]"
13025,3264,load01,13585773,5617,1.460389,"[load03, load04]"
13026,3264,load03,13585780,5624,-0.430173,"[load03, load04]"
